In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from fastai.vision import *
from projections_starter_kit import *
from audio import *
torchaudio.set_audio_backend('soundfile')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'torchaudio'

## Settings

For projections we need:
1. a model
1. a list of layers whose activations we will use
1. a data-loader

Define paths

In [6]:
path_proj = Path.cwd().parent                                        # The project directory

# datasets (audio files)
path_uw_dataset = path_proj/'datasets'/'data_uw_sounds'              # The downloaded dataset incl. audio files
path_model = path_proj/'models'/'audio_cnn_1'                        # models are saved here

In [9]:
path_model.exists(), path_uw_dataset.exists()

(True, True)

The DataLoader (Create a databunch from the audio clips in the dataset)

In [ ]:
def get_label(f):
    return f.stem.split('-')[0]

tfms=None
db_audio = (AudioList.from_folder(path=path_uw_dataset, config=cfg)
            .split_by_rand_pct(0.0)
            .label_from_func(get_label)
            .transform(tfms=tfms)
            .databunch(bs=16, num_workers = 1))

The Model - load the model we have already trained

In [ ]:
model = models.densenet161(pretrained=True).to(device)
# target_layers = [model.layer3[-2].conv2, model.layer4[-2].conv2, model.fc]